In [1]:
import os
import re
import pandas as pd
from collections import Counter
import json
import pickle

In [2]:
train_caption_path = '../data/US-Capydata-ViSportIC/normalized_dataset/train/captions.txt'
val_caption_path = '../data/US-Capydata-ViSportIC/normalized_dataset/val/captions.txt'

In [3]:
def read_captions(file_path):
    captions = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Tách đường dẫn ảnh và caption
            parts = line.strip().split('\t')
            if len(parts) == 2:
                image_path = parts[0]
                caption = parts[1]
                
                # Trích xuất tên file ảnh từ đường dẫn
                image_filename = os.path.basename(image_path)
                
                captions.append({
                    'image_path': image_path,
                    'image_filename': image_filename,
                    'caption': caption
                })
    
    return pd.DataFrame(captions)

df_train = read_captions(train_caption_path)
df_val = read_captions(val_caption_path)
caption_df = pd.concat([df_train, df_val], ignore_index=True)

In [4]:
def tokenize_caption(caption):
    tokenized_captions = []
    
    for idx, row in caption_df.iterrows():
        caption = row['caption']
        
        # Tách caption thành các token (dựa vào khoảng trắng)
        tokens = caption.strip().split()
        
        # Tạo lại caption từ tokens
        tokenized_caption = ' '.join(tokens)
        
        tokenized_captions.append(tokenized_caption)
    
    caption_df['tokenized_caption'] = tokenized_captions
    return caption_df

caption_df = tokenize_caption(caption_df)

In [5]:
def build_vocabulary(caption_df, min_word_freq=5):
    # Đếm tần suất xuất hiện của các từ
    word_counts = Counter()
    
    for caption in caption_df['tokenized_caption']:
        words = caption.split()
        word_counts.update(words)
    
    # Thêm token đặc biệt
    vocab = {
        '<pad>': 0,  # Padding
        '<unk>': 1,  # Unknown
        '<startseq>': 2,  # Bắt đầu câu 
        '<endseq>': 3  # Kết thúc câu
    }
    
    # Thêm các từ có tần suất xuất hiện lớn hơn ngưỡng
    idx = len(vocab)
    for word, count in word_counts.items():
        if count >= min_word_freq and word not in vocab:
            vocab[word] = idx
            idx += 1
    
    # Tạo mapping ngược từ index sang từ
    idx_to_word = {idx: word for word, idx in vocab.items()}
    
    print(f"Vocabulary size: {len(vocab)}")
    return vocab, idx_to_word, word_counts

vocab, idx_to_word, word_counts = build_vocabulary(caption_df, min_word_freq=5)

Vocabulary size: 1403


In [8]:
def save_vocabulary(vocab, idx_to_word, word_counts, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Lưu vocab
    with open(os.path.join(save_path, 'vocab.json'), 'w', encoding='utf-8') as f:
        json.dump(vocab, f, ensure_ascii=False, indent=2)
    
    # Lưu idx_to_word
    with open(os.path.join(save_path, 'idx_to_word.json'), 'w', encoding='utf-8') as f:
        json.dump(idx_to_word, f, ensure_ascii=False, indent=2)
    
    # Lưu word_counts
    with open(os.path.join(save_path, 'word_counts.pkl'), 'wb') as f:
        pickle.dump(dict(word_counts), f)
    
    # Lưu danh sách các từ theo tần suất xuất hiện (để phân tích)
    word_freq_df = pd.DataFrame({
        'word': list(word_counts.keys()),
        'frequency': list(word_counts.values())
    }).sort_values('frequency', ascending=False)
    
    word_freq_df.to_csv(os.path.join(save_path, 'word_frequencies.csv'), 
                         index=False, encoding='utf-8')
    
    print(f"Vocabulary saved to {save_path}")

save_vocabulary(vocab, idx_to_word, word_counts, '..\\vocabulary')

Vocabulary saved to ..\vocabulary


In [9]:
def encode_captions(caption_df, vocab):
    encoded_captions = []
    
    for caption in caption_df['tokenized_caption']:
        words = caption.split()
        # Chuyển mỗi từ thành index tương ứng trong vocab
        indices = [vocab.get(word, vocab['<unk>']) for word in words]
        encoded_captions.append(indices)
    
    caption_df['encoded_caption'] = encoded_captions
    return caption_df

caption_df = encode_captions(caption_df, vocab)

In [10]:
def create_image_caption_mapping(caption_df):
    image_to_captions = {}
    
    for idx, row in caption_df.iterrows():
        image_filename = row['image_filename']
        caption = row['tokenized_caption']
        encoded_caption = row['encoded_caption']
        
        if image_filename not in image_to_captions:
            image_to_captions[image_filename] = []
        
        image_to_captions[image_filename].append({
            'caption': caption,
            'encoded_caption': encoded_caption
        })
    
    return image_to_captions

image_caption_map = create_image_caption_mapping(caption_df)

# Lưu map
with open('../vocabulary/image_caption_map.pkl', 'wb') as f:
    pickle.dump(image_caption_map, f)

In [12]:
def analyze_vocabulary(vocab, word_counts):
    print(f"Tổng số từ trong vocabulary: {len(vocab)}")
    
    # Phân tích độ dài caption
    caption_lengths = [len(caption.split()) for caption in caption_df['tokenized_caption']]
    max_length = max(caption_lengths)
    min_length = min(caption_lengths)
    avg_length = sum(caption_lengths) / len(caption_lengths)
    
    print(f"Độ dài caption: Min = {min_length}, Max = {max_length}, Trung bình = {avg_length:.2f}")
    
    # Top 20 từ phổ biến nhất (loại bỏ <startseq> và <endseq>)
    most_common = [item for item in word_counts.most_common(22) if item[0] not in ['<startseq>', '<endseq>']][:20]
    print("Top 20 từ phổ biến nhất:")
    for word, count in most_common:
        print(f"  {word}: {count}")
    
    # Số lượng từ hiếm (xuất hiện ít hơn ngưỡng)
    rare_words = sum(1 for word, count in word_counts.items() if count < 5)
    print(f"Số lượng từ hiếm (xuất hiện < 5 lần): {rare_words}")

analyze_vocabulary(vocab, word_counts)

Tổng số từ trong vocabulary: 1403
Độ dài caption: Min = 6, Max = 33, Trung bình = 12.79
Top 20 từ phổ biến nhất:
  đang: 16614
  một: 7942
  tennis: 7629
  người: 7504
  trên: 7479
  bóng: 7268
  sân: 6139
  vận_động_viên: 4594
  cầu_thủ: 4420
  bóng_chày: 4276
  ở: 3766
  đàn_ông: 3758
  vợt: 3715
  cầm: 2966
  để: 2852
  đánh_bóng: 2717
  chơi: 2608
  trong: 2473
  và: 2214
  những: 2146
Số lượng từ hiếm (xuất hiện < 5 lần): 2310


In [14]:
!pip install h5py

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 9.7 MB/s eta 0:00:00


In [16]:
def prepare_training_data(caption_df, output_file):
    import h5py
    
    with h5py.File(output_file, 'w') as f:
        # Lưu danh sách tên file ảnh
        image_filenames = caption_df['image_filename'].tolist()
        f.create_dataset('image_filenames', data=[name.encode('utf-8') for name in image_filenames])
        
        # Lưu encoded captions
        # Padded để đảm bảo cùng độ dài
        caption_lengths = [len(cap) for cap in caption_df['encoded_caption']]
        max_length = max(caption_lengths)
        
        padded_captions = []
        for cap in caption_df['encoded_caption']:
            # Padding với 0 (<pad>) đến max_length
            padded_cap = cap + [vocab['<pad>']] * (max_length - len(cap))
            padded_captions.append(padded_cap)
        
        f.create_dataset('captions', data=padded_captions)
        f.create_dataset('caption_lengths', data=caption_lengths)
    
    print(f"Processed data saved to {output_file}")

prepare_training_data(caption_df, '../vocabulary/processed_captions.h5')

Processed data saved to ../vocabulary/processed_captions.h5
